In [1]:
import os
import pandas as pd
import glob 

In [2]:
import scipy
import numpy as np
import scipy.io as spio


def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [3]:
homedir='/media/raghuram/My Passport/dicom_seg/TCGA-LGG'

In [4]:
os.chdir(homedir)

In [5]:
dicom_file_params_df=pd.read_csv('dicom_file_params.csv')

In [17]:
dicom_file_params_df.shape

(24398, 8)

In [9]:
t1_post_samples_df = pd.read_csv('t1_post_samples.csv')
t1_pre_samples_df = pd.read_csv('t1_pre_samples.csv')
flair_samples = pd.read_csv('flair_samples.csv')
t2_pre_samples = pd.read_csv('t2_pre_samples.csv')

In [30]:
t1_post_samples_df['Sequence Name']

0       AXIAL T1 POST GAD FATSAT
1        BRAIN_CONTRAST/T1_AXIAL
2               T1 AX SE FS POST
3            AX T1 POST GD FLAIR
4             POST AX T1W_IR_TSE
5          AXIAL T1 FSE POST GAD
6               POST AX T1 FLAIR
7            MRHR T1 AX POST GAD
8                    T1 AXIAL GD
9            AX T1 MP SPGR + GAD
10                 POST AX T1 SE
11               AX T1 MP SPGR+C
12            POST AX T1 MP SPGR
13           VOLUMETRIC AXIAL GD
14               POST AX 3D SPGR
15                  AX 3D SPGR+C
16              +C AX T1 MP SPGR
17                  C+AX 3D SPGR
18                       AX T1+C
19                AX 3D SPGR + C
20                    AX T1 SE+C
21           RA ROUTINE T1 AX +C
22       +C 3D AXIAL IRSPGR FAST
23              AX T1 MP SPGR +C
24                 +C AX 3D SPGR
25    BRAIN_CONTRAST/T1_AXIAL_C+
Name: Sequence Name, dtype: object

pandas.core.series.Series

In [31]:
sequence_mapper_dict = {'T1CE':t1_post_samples_df.values,
               'T1W':t1_pre_samples_df.values,
               'T2W': t2_pre_samples.values,
               'T2F': flair_samples.values}

In [32]:
mapped_sequence = []
for idx, row in dicom_file_params_df.iterrows():
    try:
        if row['scanning_seq_mri'] in sequence_mapper_dict['T1CE']:
            mapped_sequence.append('T1CE')
        elif row['scanning_seq_mri'] in sequence_mapper_dict['T1W']:
            mapped_sequence.append('T1W')
        elif row['scanning_seq_mri'] in sequence_mapper_dict['T2W']:
            mapped_sequence.append('T2W')
        elif row['scanning_seq_mri'] in sequence_mapper_dict['T2F']:
            mapped_sequence.append('T2F')
    except Exception as e:
        print('{}, {} in row {}'.format(e, row['scanning_seq_mri'], idx))

In [33]:
len(mapped_sequence)

24398

In [35]:
dicom_file_params_df['mat_file_sequence'] = mapped_sequence

In [39]:
dicom_file_params_df.to_csv('dicom_file_params.csv')